In [ ]:
# Import the things concha might use in the notebook
import os
import sys
import math
import logging

import numpy as np
import scipy as sp
import pandas as pd

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

from concha import Planner

In [ ]:
# "Get started"
from concha import Planner

sim_planner = Planner(planner_name="sim_cafe", batch_size=8, batch_cost=24.0, unit_sale_price=4.75)

sim_planner.simulate_history(
    num_days=180,
    stockout_prob=0.5,
    demand_mean=100,
    demand_std=10,
    num_products=5
) 

sim_planner.train()
sim_planner.predict()

In [ ]:
# Set up a planner to learn from real data

from concha import Planner

planner = Planner(planner_name="this_planner", model="ProfitMaximizer")


Add the transactions .csv files to `.concha/planners/this_planner/`

In [ ]:
planner.import_transactions()
planner.update_settings()

Update the product costs, sale prices in  `.../this_planner/planner_settings.json` 

In [ ]:
planner.train()
planner.predict()

In [ ]:
planner.score_products()
planner.product_summaries

### Use the weather
Concha can ask NOAA (National Oceanic and Atmospheric Administration) for the past weather to train the
planner models, and then integrates the weather predictions to make production forecasts.
1. Get a NOAA API key [here](https://www.ncdc.noaa.gov/cdo-web/token).
 You just give them your email and they send you a key.

2. Find the nearest location tracking weather to the store/grocery/cafe.
    - Go to [Climate Data Online Search](https://www.ncdc.noaa.gov/cdo-web/search).
    - For "Select a Dataset" pick "Daily Summaries"
    - For "Enter a Search Term" put in your city/state name. Hit "SEARCH" and a map should come up.
    - On the left side of the map, above the search results and below the "Search" box,
     click on "More Search Options" and check the "Air Temperature" and "Precipitation" boxes.
     Then click "SEARCH" again.
     - When you find the nearest station, click on it and copy the `ID`. It will look like "GHCND:USC00448084"
     
3. Open the settings file for the planner in `.../concha/planners/this_planner/planner_settings.json` and
put in the NOAA API key and station ID.
    ```json
    "weather": {
        "noaa_api_key": "32pretendUEVWnoaa29Skey02",
        "noaa_station_id": "GHCND:USC00448084"
    },
    ```
   Then save the file. Now every time you run `.train()` on your model, Concha will look up the 
   weather for every date when a sale was listed and learn how weather affects demand.

In [ ]:
# Analyzing Product Performance

sim_planner = Planner(planner_name="sim_cafe", batch_size=8, batch_cost=24.0, unit_sale_price=4.75)
sim_planner.simulate_history(
    num_days=180,
    stockout_prob=0.5,
    demand_mean=100,
    demand_std=10,
    num_products=5
) 

sim_planner.score_products()
sim_planner.product_summaries


In [ ]:
# Compare model performance
sim_planner.demand_quantile = 0.1
sim_planner.grid_search(param_grid={"model":["QuantileRegressor", "Mean", "MeanWeekPart", "ProfitMaximizer"]})

In [ ]:
# View training history of deep learning models

sim_planner.plot_validation_loss()

In [ ]:
# Compare the average daily profits of each cross validation fold for all products.

sim_planner.plot_profits()

In [ ]:
# Compare the average daily waste of each cross validation fold for all products.

sim_planner.plot_wastes()